
### FastText Embeddings & Neural Network

El modelo FastText permite el enriquecimiento de vectores de palabras con información de subpalabras.

fastText trata cada palabra como la agregación de sus subpalabras. Las subpalabras se definen como los n-gramas de caracteres de la palabra. El vector de una palabra se calcula simplemente como la suma de todos los vectores de sus n-gramas de caracteres componentes.

fastText puede obtener vectores incluso para palabras fuera del vocabulario (OOV), sumando los vectores de sus char-ngrams componentes, siempre que al menos uno de los char-ngrams estuviera presente en los datos de entrenamiento.

Se implementaron los siguientes pasos:

#### 1. Preprocesamiento y Creación del corpus de palabras.

Se eliminan acentos, caracteres especiales, espacios en blanco, se normalizan numeros (12 -> '00') y se obtine una lista unica de palabras (Corpus).

#### 2. Generacion de embeddigns con FastText:

Se entrena el modelo FastText con el Corpus para obtener los embeddings por cada token:

`vectors_vocab.npy` almacena **one vector per token** (word). Las columnas corresponden a las dimensiones del vector (200 en este caso).

`vectors_ngrams` **un vector para cada n-grama** (subword) en todas los tokens del vocabulario. Cada fila es un vector que corresponde a un grupo. Las columnas corresponden a las dimensiones del vector.

FastText representa:

* `"celular"` → its own vector
* `"samsung"` → its own vector
* `"galaxy"` → its own vector
* y también sub-words `"celu"`, `"ular"`, `"sams"`, `"galax"`, etc.

El vocabulario tiene word y subwords:

```
["celular", "samsung", "galaxy", "celu", "ular", "galax", ...]
```


#### 4. Tokenización

Se utiliza el tokenizador de keras que convierte cada token en un ID. Y a una sentence en una lista de IDs

```python
sentence = "celular samsung galaxy"
```

y el tokenizar contruye este mapeo:

```python
tokenizer.word_index = {
    "celular": 1,
    "samsung": 2,
    "galaxy": 3
}
```

Luego `tokenizer.texts_to_sequences(["celular samsung galaxy"])` → `[[1, 2, 3]]`

#### 5. Construction de `embedding_matrix`

El objetivo es **alinear** el vector con el **indices del tokenizedor**, para que el modelo de ML pueda mapear *word IDs → pretrained embeddings.*

```python
for word, idx in tokenizer.word_index.items():
    if word in fasttext_model.wv:
        embedding_matrix[idx] = fasttext_model.wv[word]
```

Embedding lookup: 

Cuando introducimos la secuencia en el modelo, el modelo busca:

```
embedding_matrix[1] → vector for "celular"
embedding_matrix[2] → vector for "samsung"
embedding_matrix[3] → vector for "galaxy"
```
Se ubica por fila
```
[[v_celular],
 [v_samsung],
 [v_galaxy]]
```

Se obtiene un **2D array:** (3 words) × (200 embedding dimensions)

#### 6. Entrenamiento de la Red Neuronal 

Se utilizan los embeddings que entrenamos previamente con FastText como base.

In [ ]:
import pandas as pd
import numpy as np
import gensim
import unidecode
import spacy
from nltk.stem.snowball import SnowballStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from joblib import dump, load
import tqdm
import gc
import os
import re

def clean_spanish_text(text):
    """Limpieza básica para español: minúsculas, sin tildes, sin caracteres raros."""
    text = str(text).lower().strip()
    text = unidecode.unidecode(text)  # remove accents
    text = re.sub(r'\d', '0', text)    # normalize numbers
    text = re.sub(r'[^a-z0-9ñ ]', ' ', text)  # keep only letters, numbers, ñ
    text = re.sub(r'\s+', ' ', text).strip() # remove extra spaces
    return text

def build_corpus_from_dataframe(df):
    """Crea una lista de listas de palabras a partir del dataframe (columna 'text')."""
    df['text'] = df['text'].apply(clean_spanish_text)
    return df['text'].str.split().tolist()

def train_fasttext_es(corpus, output_dir="models_fasttext_es", 
                      vector_size=200, window=8, min_count=5, workers = 4):
    """Entrena y guarda un modelo FastText en español."""
    print("🧠 Entrenando modelo FastText (es)...")
    model = gensim.models.FastText(
        sentences=corpus,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        sg=1,
        workers=workers
    )
    os.makedirs(output_dir, exist_ok=True)
    model.save(os.path.join(output_dir, "fasttext_es.bin"))
    print("✅ Modelo FastText guardado en:", output_dir)
    return model

def build_embedding_matrix_es(model_path, tokenizer_path, output_dir="models_fasttext_2_es"):
    """Crea la matriz de embeddings alineada con el tokenizer (solo español)."""
    print("⚙️ Construyendo matriz de embeddings (es)...")

    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "embedding_matrix_es.npy")

    # Si ya existe un embedding_matrix guardado, lo reutiliza
    if os.path.exists(output_path):
        print(f"🔁 Archivo existente encontrado: {output_path}")
        print("Cargando matriz desde disco...")
        return np.load(output_path)

    # Carga modelo FastText y tokenizer
    model = gensim.models.FastText.load(model_path)
    word_vectors = model.wv
    tokenizer = load(tokenizer_path)

    nlp = spacy.load("es_core_news_sm", disable=['parser', 'ner'])
    stemmer = SnowballStemmer(language='spanish')

    # Inicializa la matriz (una fila por palabra del tokenizer x vector size (200))
    embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, word_vectors.vector_size))
    unknown = 0

    # Construye los embeddings
    for word, idx in tqdm.tqdm(tokenizer.word_index.items()):
        vector = None
        # Crea un FastText vector por cada word in keras.tokenizer.
        # Si la word no existe en los word_vectors, busca por sus variantes:
        #   > unidecode(), lemma_ or stemmer
        # Ante el primer match entre word/variante & word_vectors, se sale del bucle
        for variant in [word, unidecode.unidecode(word), nlp(word)[0].lemma_, stemmer.stem(word)]:
            if variant in word_vectors:
                vector = word_vectors[variant]
                break
        # agrega el vector a la matriz
        if vector is not None:
            embedding_matrix[idx] = vector
        else:
            unknown += 1

    print(f"🔍 Palabras desconocidas: {unknown}")
    print(f"💾 Guardando matriz en {output_path} ...")

    np.save(output_path, embedding_matrix)
    print("✅ Embedding matrix guardada correctamente.")
    
    return embedding_matrix


In [2]:
df_all = pd.read_csv('train.csv')

In [3]:
df_all.head()

,title,label_quality,language,category
0,Hidrolavadora Lavor One 120 Bar 1700w Bomba A...,unreliable,spanish,ELECTRIC_PRESSURE_WASHERS
1,Placa De Sonido - Behringer Umc22,unreliable,spanish,SOUND_CARDS
2,Maquina De Lavar Electrolux 12 Kilos,unreliable,portuguese,WASHING_MACHINES
3,Par Disco De Freio Diant Vent Gol 8v 08/ Frema...,unreliable,portuguese,VEHICLE_BRAKE_DISCS
4,Flashes Led Pestañas Luminoso Falso Pestañas P...,unreliable,spanish,FALSE_EYELASHES


In [4]:
df_sample = df_all[df_all['language'] == 'spanish']
df_sample = df_sample.rename(columns = {'title': 'text', 'category' : 'labels'})
df_sample = df_sample[['text', 'labels']]

In [7]:
df_sample.head()

,text,labels
0,Hidrolavadora Lavor One 120 Bar 1700w Bomba A...,ELECTRIC_PRESSURE_WASHERS
1,Placa De Sonido - Behringer Umc22,SOUND_CARDS
4,Flashes Led Pestañas Luminoso Falso Pestañas P...,FALSE_EYELASHES
9,Gatito Lunchera Neoprene,LUNCHBOXES
11,Rosario Contador De Billetes Uv / Mg Detecta F...,BILL_COUNTERS


In [8]:
len(df_sample)

10000000

In [ ]:
# min_samples = 20
# valid_labels = df_sample['labels'].value_counts()
# valid_labels = valid_labels[valid_labels >= min_samples].index
# df_sample = df_sample[df_sample['labels'].isin(valid_labels)]

In [13]:
df_sample.iloc[[0,1]]

,text,labels
0,Hidrolavadora Lavor One 120 Bar 1700w Bomba A...,ELECTRIC_PRESSURE_WASHERS
1,Placa De Sonido - Behringer Umc22,SOUND_CARDS


In [15]:
df_sample_filter = df_sample.iloc[[0,1]]
build_corpus_from_dataframe(df_sample_filter)

C:\Users\maribel.fraire\AppData\Local\Temp\ipykernel_10532\7008074.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(clean_spanish_text)


[['hidrolavadora',
  'lavor',
  'one',
  '000',
  'bar',
  '0000w',
  'bomba',
  'aluminio',
  'italia'],
 ['placa', 'de', 'sonido', 'behringer', 'umc00']]

In [ ]:
# 1️⃣ Crear corpus
# Apply clean_spanish_text to df_sample['text'] and create corpus
corpus = build_corpus_from_dataframe(df_sample)

In [19]:
len(corpus)

10000000

In [ ]:
# 2️⃣ Entrenar modelo FastText español
fasttext_model = train_fasttext_es(corpus, output_dir="models_fasttext_2_es")

🧠 Entrenando modelo FastText (es)...
✅ Modelo FastText guardado en: models_fasttext_2_es


In [21]:
from gensim.models import FastText

fasttext_model = FastText.load("models_fasttext_2_es/fasttext_es.bin")

In [22]:
# 3️⃣ Crear y guardar tokenizer
tokenizer = Tokenizer()
# df_sample['text'] is already cleaned at this point
tokenizer.fit_on_texts(df_sample['text'])
dump(tokenizer, "models_fasttext_2_es/tokenizer_es.dump")

['models_fasttext_2_es/tokenizer_es.dump']

In [27]:
print("Total words in tokenizer:", len(tokenizer.word_index))
for word, idx in list(tokenizer.word_index.items())[:10]:
    print(f'{word}: {idx}')

Total words in tokenizer: 658651
0: 1
de: 2
00: 3
000: 4
0000: 5
para: 6
x: 7
con: 8
y: 9
a: 10


In [30]:
# 4️⃣ Generar embedding matrix 
embedding_matrix = build_embedding_matrix_es(model_path = "models_fasttext_2_es/fasttext_es.bin", 
                                             tokenizer_path= "models_fasttext_2_es/tokenizer_es.dump",
                                             output_dir = "models_fasttext_2_es")


⚙️ Construyendo matriz de embeddings (es)...


100%|██████████| 658651/658651 [19:40<00:00, 557.91it/s]


🔍 Palabras desconocidas: 0
💾 Guardando matriz en models_fasttext_2_es\embedding_matrix_es.npy ...
✅ Embedding matrix guardada correctamente.


In [43]:
#Get id for celular
id_celular = tokenizer.word_index['celular']
print(f'id_celular: {id_celular}')

#Get embedding for celular
print(embedding_matrix[id_celular][:10])
print(f'len: {len(embedding_matrix[id_celular])}')

id_celular: 506
[-0.02472354  0.09676356  0.13969924 -0.05794974 -0.06126994  0.11244962
 -0.08809794  0.1878252  -0.06249603 -0.19332647]
len: 200


In [44]:
embedding_matrix.shape

(658652, 200)

In [45]:
df_sample['labels'].value_counts()

labels
BOOKS                          19010
ACTION_FIGURES                 18433
MAGAZINES                      18081
DIECAST_VEHICLES               17923
FOOTBALL_SHIRTS                17923
                               ...  
SCALE_RULERS                      49
COMMERCIAL_POPCORN_MACHINES       36
SNACK_HOLDERS                      9
ANTI_STATIC_PLIERS                 5
CARD_PAYMENT_TERMINALS             2
Name: count, Length: 1574, dtype: int64

In [46]:
df_tablets = df_sample['text'][df_sample['labels'] == 'TABLETS']
for i in range(10):
    print(df_tablets.iloc[i])

se vende galaxy tab0 0 0 wi fi
tablet avh g00 funda con teclado
ipad mini 0 retina wifi 00gb inmaculado
ipad a 0000 0 00 gb wifi con funda lapiz y cable de datos
samsung galaxy tab s0 0 0 00gb igual a nueva
ipad air 0 00gb como nuevo con funda y accesorios
tablet samsung note 00 0 wifi
tablet philco tp0a0 0 kids blanco 0 gb funda naranja
aire 0 caso robusta pata de cabra serie a prueba de
tablet 0 0 pulgadas lenovo


### Word vector lookup

Toda la información necesaria para buscar palabras fastText (incl. OOV words) se encuentra en su atributo `model.wv.`

In [47]:
fasttext_model.wv.most_similar("moto", topn=5)

[('moto0', 0.7987481951713562),
 ('motoe0', 0.7944773435592651),
 ('motox', 0.7801886200904846),
 ('moto00', 0.7801306247711182),
 ('motoxwilde', 0.7686169743537903)]

Previamente, utilicé estos hyperparameters para entrenar los embeddings en models.FastText() 

vector_size=200, window=5, min_count=2

Mi objetivo era que esto fasttext_model.wv.most_similar_cosmul( positive=["galaxy"], negative=["apple"], topn=5 ) retorne "samsung", sin embargo este fue el resultado: 

```PYTHON
[('63', 1.0004220008850098),
('3v', 1.0004171133041382),
('rw', 1.0003960132598877),
('84', 1.0002875328063965),
('h/', 1.00026273727417)]
```

Parece el los numeros o junk son dominantes y Se excedió en la generación de Character n-grams 

Al cambiar los hiperparametros a `window = 8` y `min_count = 5` el resultado tiene mas sentido. Al aumentar el min_count de 2 a 5, hace que se ignoren esos valores 'raros' (numeros, ngramas) ya que ahora le pido que tengan al menos 5 apariciones.

- window: Context window size (Default 5)
- min_count: Ignore words with number of occurrences below this (Default 5)


[Doc](https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html)

In [48]:
print(fasttext_model.wv.most_similar("galaxy", topn=5))
print(fasttext_model.wv.most_similar("tablet", topn=5))
print(fasttext_model.wv.most_similar("apple", topn=5))

[('galax', 0.833404541015625), ('galaxi', 0.8131084442138672), ('galaxis', 0.7421173453330994), ('galaxie', 0.6850820183753967), ('j0', 0.6780985593795776)]
[('tablets', 0.8222858905792236), ('phablet', 0.7704592943191528), ('inchtablet', 0.7572232484817505), ('ablet', 0.7494990229606628), ('tableto', 0.7379324436187744)]
[('applea', 0.8668965697288513), ('appletv', 0.8461295962333679), ('apples', 0.8112044930458069), ('applewatch', 0.7704615592956543), ('applecare', 0.7282817363739014)]


In [59]:
fasttext_model.wv.most_similar_cosmul(
    positive=["tab0", "samsung"],
    negative=["lenovo"],
    topn=3
)

[('smsung', 0.9951120018959045),
 ('galaxy', 0.9924536347389221),
 ('samsumg', 0.9871987700462341)]

In [60]:
fasttext_model.wv.most_similar_cosmul(
    positive=["tab0", "samsung"],
    negative=["apple"],
    topn=3
)

[('s0msung', 1.0625591278076172),
 ('smsung', 1.0573632717132568),
 ('sanmsung', 1.0518909692764282)]

In [50]:
fasttext_model.wv.most_similar_cosmul(positive=['amd', 'i0'], negative=['intel'], topn=3)

[('ryzen', 0.9222611784934998),
 ('ryzen0', 0.9118472337722778),
 ('fryzen', 0.8881929516792297)]

Antes del preprocesamiento y la normalizacion de los numeros '2025' -> '0000' '8' -> '0' la embedding_matrix era de (1228979, 200) **Se redujo más de la mitad**

In [62]:
embedding_matrix.shape

(658652, 200)

keras.tokenizer transforma cada token en un ID

tokenizer.texts_to_sequences transforma una secuencia de palabras en una sequencia de tokens

In [63]:
tokenizer.texts_to_sequences(["Celular Samsung"])

[[506, 80]]

In [65]:
#vector for celular    vector for samsung
embedding_matrix[506][:10], embedding_matrix[80][:10]

(array([-0.02472354,  0.09676356,  0.13969924, -0.05794974, -0.06126994,
         0.11244962, -0.08809794,  0.1878252 , -0.06249603, -0.19332647]),
 array([ 0.03803588,  0.31229076,  0.16416894,  0.47289488, -0.08413718,
        -0.19058801, -0.40931073, -0.28712058, -0.35503694, -0.27083719]))

### Neural Network

Encode labels. Convertir de STRING a ID

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_sample['label_id'] = le.fit_transform(df_sample['labels'])
df_sample[['labels', 'label_id']].head()

,labels,label_id
0,ELECTRIC_PRESSURE_WASHERS,539
1,SOUND_CARDS,1304
4,FALSE_EYELASHES,599
9,LUNCHBOXES,906
11,BILL_COUNTERS,182


#### Load Tokenizer and FastText Model

In [ ]:
# import pickle
# from gensim.models import FastText
# # Load tokenizer
# with open("models_fasttext_es/tokenizer_es.dump", "rb") as f:
#     tokenizer = pickle.load(f)
# # Load FastText model
# fasttext_model = FastText.load("models_fasttext_es/fasttext_es.bin")

In [67]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = tokenizer.texts_to_sequences(df_sample["text"])
X = pad_sequences(X, maxlen=None, padding='post')
y = df_sample["label_id"].values

In [68]:
X[:3]

array([[  823, 34333,   344,     4,   357,   128,    51,    86,  2342,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0],
       [   94,     2,   531,  1772, 24769,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0],
       [24017,    15,  1623,  2976,  4998,  1623,     6,  4406,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0]],
      dtype=int32)

In [69]:
y[:3], df_sample['labels'].head(3)

(array([ 539, 1304,  599]),
 0    ELECTRIC_PRESSURE_WASHERS
 1                  SOUND_CARDS
 4              FALSE_EYELASHES
 Name: labels, dtype: object)

In [70]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = embedding_matrix.shape[1]
num_classes = len(np.unique(y))

model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        input_length=50,
        trainable=False  # keep FastText vectors fixed
    ),
    GlobalAveragePooling1D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=1e-3),
    metrics=['accuracy']
)


c:\Users\maribel.fraire\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [71]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [72]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=128,
    verbose=1
)

Epoch 1/10
62500/62500 ━━━━━━━━━━━━━━━━━━━━ 509s 8ms/step - accuracy: 0.6163 - loss: 1.7396 - val_accuracy: 0.8049 - val_loss: 0.8887
Epoch 2/10
62500/62500 ━━━━━━━━━━━━━━━━━━━━ 540s 9ms/step - accuracy: 0.6712 - loss: 1.4466 - val_accuracy: 0.8106 - val_loss: 0.8706
Epoch 3/10
62500/62500 ━━━━━━━━━━━━━━━━━━━━ 581s 9ms/step - accuracy: 0.6764 - loss: 1.4267 - val_accuracy: 0.8119 - val_loss: 0.8634
Epoch 4/10
62500/62500 ━━━━━━━━━━━━━━━━━━━━ 641s 10ms/step - accuracy: 0.6796 - loss: 1.4133 - val_accuracy: 0.8127 - val_loss: 0.8598
Epoch 5/10
62500/62500 ━━━━━━━━━━━━━━━━━━━━ 637s 10ms/step - accuracy: 0.6815 - loss: 1.4056 - val_accuracy: 0.8133 - val_loss: 0.8556
Epoch 6/10
62500/62500 ━━━━━━━━━━━━━━━━━━━━ 644s 10ms/step - accuracy: 0.6829 - loss: 1.3998 - val_accuracy: 0.8145 - val_loss: 0.8528
Epoch 7/10
62500/62500 ━━━━━━━━━━━━━━━━━━━━ 627s 10ms/step - accuracy: 0.6838 - loss: 1.3953 - val_accuracy: 0.8134 - val_loss: 0.8537
Epoch 8/10
62500/62500 ━━━━━━━━━━━━━━━━━━━━ 625s 10ms/step

La métrica de **accuracy en validación es de 0.81**. 

Previamente, se probó el modelo en el dataset sin eliminar caracteres especiales/puntuación y sin normalizar los numeros y se obtubo 0.79 de accuracy

In [73]:
model.save("models_fasttext_2_es/text_classifier_es.h5")

In [ ]:
# from tensorflow.keras.models import load_model

# model = load_model("models_fasttext_2_es/text_classifier_es.h5")